## 38 Renderizando parte de uma textura

As imagens usadas neste notebook foram obtidas nos sites:

https://www.gratispng.com/

https://www.subpng.com/

https://www.cleanpng.com/

In [2]:
import glm
import time
import numpy as np
import OpenGL.GL as gl
from PyQt5 import QtOpenGL
from PyQt5.QtCore import QCoreApplication
from PyQt5.QtWidgets import QApplication

from cg.shader_programs.SimpleShaderProgram_v3 import SimpleShaderProgram
from cg.renderers.ModelRenderer_v4 import ModelRenderer
from cg.models.SquareMesh_v3 import SquareMesh
from cg.utils.Texture import Texture

class MyWidget(QtOpenGL.QGLWidget):
    def initializeGL(self):
        
        self.cameraPos   = glm.vec3(0.0, 0.0, 1.2)
        self.cameraFront = glm.vec3(0.0, 0.0, 0.0)
        self.cameraUp    = glm.vec3(0.0, 1.0,  0.0)
    
        self.viewMatrix = glm.lookAt(self.cameraPos, self.cameraFront, self.cameraUp)
        
        # define uma malha quadrada
        vertex_position = np.array([
            -0.50, -0.50, 0.0, 1.0, # Triângulo 1
             0.50, -0.50, 0.0, 1.0,
            -0.50,  0.50, 0.0, 1.0,
             0.50, -0.50, 0.0, 1.0, # Triângulo 2
             0.50,  0.50, 0.0, 1.0,
            -0.50,  0.50, 0.0, 1.0],
            dtype=np.float32)
        
        # define as coordendas de texturas para renderizá-la por completo
        self.texCoord = np.array([
             0.00,  1.00, # Triângulo 1
             1.00,  1.00,
             0.00,  0.00,
             1.00,  1.00, # Triângulo 2
             1.00,  0.00,
             0.00,  0.00],
            dtype=np.float32)
        
        # calcula o tamanho de uma parte/quadro/frame da textura
        # as texturas carregadas possuem frames com o mesmo tamanho
        frame_size = np.array([1.0 / 4, 1.0 / 3], dtype=np.float32)
        self.texCoordFirstFrame = self.texCoord.reshape((6, 2)) * frame_size
        
        # calcula o offset dos outros frames em relação ao primeiro frame
        x = np.linspace(0, 1, 5)
        y = np.linspace(0, 1, 4)
        self.frameOffset_x, self.frameOffset_y = np.meshgrid(x[:-1], y[:-1])
        
        self.frameOffset_x = self.frameOffset_x.reshape((12)).astype(np.float32)
        self.frameOffset_y = self.frameOffset_y.reshape((12)).astype(np.float32)
        
        # carrega e cria as texturas
        self.texture_1 = Texture('./cg/images/textures/skybox.png')
        self.texture_2 = Texture('./cg/images/textures/skybox2.png')
        self.texture_3 = Texture('./cg/images/textures/skybox3.png')
        
        # cria o objeto responsável por carregar os dados para a GPU e renderizá-los
        self.squareRenderer = ModelRenderer(vertex_position, vertex_tex=self.texCoord)
        
        # cria um shader program simples
        self.shaderProgram = SimpleShaderProgram()
    
        # recupera o endereço da variável de entrada do shader program
        position_loc = self.shaderProgram.getVertexPositionLoc()
        tex_coord_loc = self.shaderProgram.getVertexTextureCoordLoc()
        
        # configura os dados do modelo para serem os dados de entrada do shader program
        self.squareRenderer.setVertexPositionLoc(position_loc)
        self.squareRenderer.setVertexTextureCoordLoc(tex_coord_loc)
        
        # configura a cor de background
        gl.glClearColor(0.9, 0.9, 0.9, 1)
        
        # habilita teste de profundidade e culling
        gl.glEnable(gl.GL_DEPTH_TEST);
        gl.glEnable(gl.GL_CULL_FACE)
        
    def paintGL(self):
            
        # limpa o background com a cor especificada e o buffer de profundidade
        gl.glClear(gl.GL_COLOR_BUFFER_BIT | gl.GL_DEPTH_BUFFER_BIT)
        
        # ativa o shader program que será executado pela GPU
        self.shaderProgram.bind()
        
        # renderiza os quadrados texturizados
        self.renderTexture1()
        self.renderTexture2()
        self.renderTexture3()
        
        # desativa o shader program
        self.shaderProgram.release()
        
        # solicita que o método paintGL seja chamado novamente
        self.update()
    
    def renderTexture1(self):
        
        # calcula as matrizes de transformação do quadrado e atualiza a matriz do shader
        model_matrix = glm.translate(glm.mat4(), glm.vec3(-0.7, 0.0, -1.5))
        self.shaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        # configura/ativa a textura a ser usada (e desativa o uso da cor uniforme)
        self.shaderProgram.bindTexture2D(self.texture_1.getTextureID())
        
        # atualiza as coordenadas de textura para renderiza a textura toda
        self.squareRenderer.updateVertexTextureCoord(self.texCoord)
        
        # renderiza o quadrado usando toda a textura
        self.squareRenderer.render()
        
        # calcula as matrizes de transformação do quadrado e atualiza a matriz do shader
        model_matrix = glm.translate(glm.mat4(), glm.vec3(0.7, 0.0, -1.5))
        self.shaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        # calcula as coordenadas do frame desejado
        frame_index = 6
        offset_x = self.frameOffset_x[frame_index]
        offset_y = self.frameOffset_y[frame_index]
        frame_tex = self.texCoordFirstFrame + np.array([offset_x, offset_y])

        # atualiza a GPU com as coordenadas de textura do frame desejado
        self.squareRenderer.updateVertexTextureCoord(frame_tex)
        
        # renderiza o quadrado a parte da textura desejada
        self.squareRenderer.render()
        
        # desativa a textura (e ativa o uso da cor uniforme)
        self.shaderProgram.releaseTexture2D()
        
    def renderTexture2(self):
        
        # calcula as matrizes de transformação do quadrado e atualiza a matriz do shader
        model_matrix = glm.translate(glm.mat4(), glm.vec3(-0.7, 1.05, -1.5))
        self.shaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        # configura/ativa a textura a ser usada (e desativa o uso da cor uniforme)
        self.shaderProgram.bindTexture2D(self.texture_2.getTextureID())
        
        # atualiza as coordenadas de textura para renderiza a textura toda
        self.squareRenderer.updateVertexTextureCoord(self.texCoord)
        
        # renderiza o quadrado usando toda a textura
        self.squareRenderer.render()

        # calcula as matrizes de transformação do quadrado e atualiza a matriz do shader
        model_matrix = glm.translate(glm.mat4(), glm.vec3(0.7, 1.05, -1.5))
        self.shaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        # calcula as coordenadas do frame desejado
        frame_index = 9
        offset_x = self.frameOffset_x[frame_index]
        offset_y = self.frameOffset_y[frame_index]
        frame_tex = self.texCoordFirstFrame + np.array([offset_x, offset_y])
        
        # atualiza a GPU com as coordenadas de textura do frame desejado
        self.squareRenderer.updateVertexTextureCoord(frame_tex)
        
        # renderiza o quadrado com a parte da textura desejada
        self.squareRenderer.render()
        
        # desativa a textura (e ativa o uso da cor uniforme)
        self.shaderProgram.releaseTexture2D()
    
    def renderTexture3(self):
        
        # calcula as matrizes de transformação do quadrado e atualiza a matriz do shader
        model_matrix = glm.translate(glm.mat4(), glm.vec3(-0.7, -1.05, -1.5))
        self.shaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        # configura/ativa a textura a ser usada (e desativa o uso da cor uniforme)
        self.shaderProgram.bindTexture2D(self.texture_3.getTextureID())
        
        # atualiza as coordenadas de textura para renderiza a textura toda
        self.squareRenderer.updateVertexTextureCoord(self.texCoord)
        
        # renderiza o quadrado usando toda a textura
        self.squareRenderer.render()

        # calcula as matrizes de transformação do quadrado e atualiza a matriz do shader
        model_matrix = glm.translate(glm.mat4(), glm.vec3(0.7, -1.05, -1.5))
        self.shaderProgram.setUniformMVPMatrix(self.perspectiveMatrix * self.viewMatrix * model_matrix)
        
        # calcula as coordenadas do frame desejado
        frame_index = 4
        offset_x = self.frameOffset_x[frame_index]
        offset_y = self.frameOffset_y[frame_index]
        frame_tex = self.texCoordFirstFrame + np.array([offset_x, offset_y])
        
        # atualiza a GPU com as coordenadas de textura do frame desejado
        self.squareRenderer.updateVertexTextureCoord(frame_tex)
        
        # renderiza o quadrado com a parte da textura desejada
        self.squareRenderer.render()
        
        # desativa a textura (e ativa o uso da cor uniforme)
        self.shaderProgram.releaseTexture2D()

    def resizeGL(self, width, height):
        gl.glViewport(0, 0, width, height)
        
        # configura a projeção
        aspectRatio = width / height
        self.perspectiveMatrix = glm.perspective(glm.radians(60.0), aspectRatio, 0.1, 50.0)

def main():
    import sys

    #Criação de um aplicativo Qt
    app = QCoreApplication.instance()
    if app is None:
        app = QApplication(sys.argv)

    #Especificação do contexto OpenGL
    glformat = QtOpenGL.QGLFormat()
    glformat.setVersion(3, 3)
    glformat.setDoubleBuffer(True)
    glformat.setProfile(QtOpenGL.QGLFormat.CoreProfile)
    
    #Criação da janela de renderização
    w = MyWidget(glformat)
    w.resize(900, 800)
    w.setWindowTitle('OpenGL example')
    w.show()
    
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

trying to open ./cg/images/textures/skybox.png
opened file: size= (1024, 768) format= PNG mode= RGBA
trying to open ./cg/images/textures/skybox2.png
opened file: size= (1024, 768) format= PNG mode= RGBA
trying to open ./cg/images/textures/skybox3.png
opened file: size= (1024, 768) format= PNG mode= RGBA


SystemExit: 0

C:\Users\Rapha\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [20]:
! jupyter nbconvert --to python 38_Renderizando_parte_de_uma_textura.ipynb
%run -i 38_Renderizando_parte_de_uma_textura.py

[NbConvertApp] Converting notebook 37_Quadrado_com_texturas.ipynb to python
[NbConvertApp] Writing 8073 bytes to 37_Quadrado_com_texturas.py


trying to open ./pngwing.com_1.png
opened file: size= (650, 650) format= None mode= RGBA
trying to open ./pngwing.com_2.png
opened file: size= (1000, 1000) format= None mode= RGBA
trying to open ./pngwing.com_4.png
opened file: size= (650, 650) format= None mode= RGBA
